In [1]:
import cudf
import dask_cudf
import cugraph
import cuspatial
import math
import geopy
import geopandas as gpd
from shapely.geometry import Point
from dask import delayed
import json
from geopy.geocoders import Nominatim
import dash_leaflet as dl
geolocator = Nominatim(user_agent="test_app")
DATA_DIR = "../data/"

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_leaflet.express as dlx
import pandas as pd
from dash_extensions.javascript import Namespace, arrow_function
from dash.dependencies import Input, Output

In [3]:
census_data = dask_cudf.read_parquet('../../plotly/plotly-dash-rapids-census-demo/data/census_data.parquet/')
nodes_df = cudf.read_parquet(f'{DATA_DIR}us-nodes.parquet')
edges_df = cudf.read_parquet(f'{DATA_DIR}us-edges.parquet')

In [4]:
nodes_df.head(1000).to_parquet(f'{DATA_DIR}us-nodes-lite.parquet')

In [5]:
edges_df.head(1000).to_parquet(f'{DATA_DIR}us-edges-lite.parquet')

In [4]:
type(census_data)

dask_cudf.core.DataFrame

In [6]:
isinstance(census_data, dask_cudf.core.DataFrame)

True

In [4]:
def createCircleAroundWithRadius(lat, lon, radiusMiles):
    latArray = []
    lonArray = []

    for brng in range(0,360):
        lat2, lon2 = getLocation(lat,lon,brng,radiusMiles)
        latArray.append(lat2)
        lonArray.append(lon2)

    return lonArray,latArray


def getLocation(lat1, lon1, brng, distanceMiles):
    lat1 = lat1 * math.pi/ 180.0
    lon1 = lon1 * math.pi / 180.0
    #earth radius
    #R = 6378.1Km
    #R = ~ 3959 MilesR = 3959
    R = 3959

    distanceMiles = distanceMiles/R

    brng = (brng / 90)* math.pi / 2

    lat2 = math.asin(math.sin(lat1) * math.cos(distanceMiles) 
    + math.cos(lat1) * math.sin(distanceMiles) * math.cos(brng))

    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(distanceMiles)
    * math.cos(lat1),math.cos(distanceMiles)-math.sin(lat1)*math.sin(lat2))

    lon2 = 180.0 * lon2/ math.pi
    lat2 = 180.0 * lat2/ math.pi

    return lat2, lon2

In [5]:
def get_updated_df(lat,lon, nodes_df):
#     nodes_df = cudf.read_parquet(f'{DATA_DIR}us-nodes.parquet')
    results = cuspatial.point_in_polygon(nodes_df.x, nodes_df.y, cudf.Series([0], index=["selection"]), [0], lat, lon)
    return nodes_df[results.selection]

def get_updated_edges(nodes, edges):
#     edges = cudf.read_parquet(f'{DATA_DIR}us-edges.parquet')
    indices = cudf.logical_and(
        edges.src.isin(nodes.vertex), edges.dst.isin(nodes.vertex),
    )
    return edges[indices]

def get_shortest_paths(edges_df, point_of_interest):
    G_gpu = cugraph.Graph()
    G_gpu.from_cudf_edgelist(edges_df, source='src', destination='dst', edge_attr='time')
    shortest_paths = cugraph.traversal.sssp(G_gpu, point_of_interest)
    shortest_paths = shortest_paths.drop('predecessor', axis=1)
    shortest_paths.columns = ['time', 'vertex']
    return shortest_paths

def get_nearest_node(gdf, point, x='x', y='y', osmid='osmid'):
    gdf = gdf
    gdf['point_y'] = point[0
    @app.callback(
        [
            Output('indicator-graph', 'figure'),
            Output("layer", "children"), Output("polygons", "data"),
            Output("scatter-population", "data"),

            # Output('education-histogram', 'figure'),
            # Output('income-histogram', 'figure'),
            # Output('cow-histogram', 'figure'),
            # Output('age-histogram', 'figure'),

            # Output('education-histogram', 'config'),
            # Output('income-histogram', 'config'),
            # Output('cow-histogram', 'config'),
            # Output('age-histogram', 'config')
        ],
        [
            Input("map-graph", "click_lat_lng"), Input('average-speed', 'value'),
            Input('colorscale-dropdown', 'value'), Input('gpu-toggle', 'on')
        ]
    )
    def update_plots(
            click_lat_lng, average_speed, colorscale_name, gpu_enabled
    ):
        global census_data, cudf_nodes, cudf_edges

        t0 = time.time()

        if click_lat_lng is not None:
            lat, lon = click_lat_lng
            marker = dl.Marker(position=click_lat_lng, children=dl.Tooltip("({:.3f}, {:.3f})".format(*click_lat_lng)))

            polygons, df = get_nearest_polygons_from_selected_point(lat, lon, average_speed, cudf_nodes, cudf_edges, census_data)
            polygon_data = json.loads(polygons.to_json())
            scatter_data = get_data(df.head(200_000))
        else:
            marker, polygon_data, scatter_data, df = None, None, None, None

        if df == None:
            len_df = len(census_data)
        else:
            len_df = len(df)
        # if isinstance(df_d, dask_cudf.core.DataFrame):
        #     df_d = df_d.compute()

        # figures_d = delayed(build_updated_figures)(df_d, colorscale_name)

        # figures = figures_d.compute()

        # (education_histogram, income_histogram,
        #  cow_histogram, age_histogram, n_selected_indicator) = figures

        # barchart_config = {
        #     'displayModeBar': True,

        #     'modeBarButtonsToRemove': [
        #         'zoom2d', 'pan2d', 'select2d', 'lasso2d', 'zoomIn2d', 'zoomOut2d',
        #         'resetScale2d', 'hoverClosestCartesian', 'hoverCompareCartesian', 'toggleSpikelines'
        #     ]
        # }
        n_selected_indicator = {
            'data': [{
                'domain': {
                    'x': [0, 0.5], 'y': [0, 0.5]
                },
                'title': {'text': 'Query Result'},
                'type': 'indicator',
                'value': len_df,
                'number': {
                    'font': {
                        'color': text_color,
                        'size': '50px'
                    },
                    "valueformat": ","
                }
            }],
            'layout': {
                'template': template,
                'height': row_heights[3],
                'margin': {'l': 10, 'r': 10, 't': 10, 'b': 10}
            }
        }
        compute_time = time.time() - t0
        print(f"Update time: {compute_time}")
        n_selected_indicator['data'].append({
            'title': {"text": "Query Time"},
            'type': 'indicator',
            'value': round(compute_time, 4),
            'domain': {'x': [0.51, 1], 'y': [0, 0.5]},
            'number': {
                'font': {
                    'color': text_color,
                    'size': '50px'
                },
                'suffix': "s"
            }
        })
        return (
            n_selected_indicator,
            marker, polygon_data, scatter_data,
            # education_histogram, income_histogram, cow_histogram, age_histogram,
            # barchart_config, barchart_config, barchart_config, barchart_config,
        )
]
    gdf['point_x'] = point[1]
    gdf['distance'] = cuspatial.haversine_distance(gdf[y], gdf[x], 
                                                             gdf['point_y'], gdf['point_x'])
    mask = gdf['distance'] == gdf['distance'].min()
    nearest_node = gdf[mask][osmid].values[0]
    gdf.drop(['point_y', 'point_x', 'distance'], axis=1, inplace=True)
    return nearest_node

def get_polygons_for_travel_time(results, trip_times):
    for trip_time in trip_times:
        results['within_' + str(trip_time)] = 1.0 * (results['time'] < trip_time)

    # make the isochrone polygons
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        mask = results['within_' + str(trip_time)] == 1
        subset = results[mask].to_pandas()
        node_points = gpd.points_from_xy(subset.x, subset.y)
        bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull  # TODO use cuspatial
        isochrone_polys.append(bounding_poly)
    return isochrone_polys

from pyproj import Proj, Transformer

In [6]:
def query_census_dataset(polygons, census_data):
    final_polygon = polygons[0]
    for i in polygons:
        final_polygon = final_polygon.union(i)
    lat, lon = final_polygon.exterior.coords.xy
    transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
    lon, lat = transform_4326_to_3857.transform(lon, lat)
    results = cuspatial.point_in_polygon(census_data.x, census_data.y, cudf.Series([0], index=["selection"]), [0], lon, lat)
    return census_data[results.selection]

In [7]:
def get_data(df, color_prop="income"):
    print(df)
    if len(df)>0:
        transform_3857_to_4326 = Transformer.from_crs('epsg:3857', 'epsg:4326')
        df['lat'], df['lon'] = transform_3857_to_4326.transform(df.x.to_array(), df.y.to_array())
        df = df[['lat', 'lon', 'sex', 'education', 'income']]  # drop irrelevant columns
        if isinstance(df, cudf.DataFrame):
            dicts = df.to_pandas().to_dict('rows')
        else:
            dicts = df.to_dict('rows')
        for item in dicts:
            item["tooltip"] = "{:.1f}".format(item[color_prop])  # bind tooltip
            item["popup"] = item["income"]  # bind popup
        geojson = dlx.dicts_to_geojson(dicts, lat="lat",lon="lon")  # convert to geojson
#         geobuf = dlx.geojson_to_geobuf(geojson)  # convert to geobuf
        return geojson
    return ''

In [8]:
distanceInMiles = 30
trip_times = [5, 10, 15, 30, 60]

def get_nearest_polygons_from_selected_point(point_lat, point_lon, average_speed):
    lat, lon =  createCircleAroundWithRadius(point_lat, point_lon, distanceInMiles)
    nodes = get_updated_df(lat, lon, nodes_df)
    edges = get_updated_edges(nodes, edges_df)
    meters_per_minute = average_speed * 1000 / 60 # km per hour to m per minute
    edges['time'] = edges['length'] / meters_per_minute
    point_of_interest = get_nearest_node(nodes, point=(point_lat,point_lon), x='x', y='y', osmid='vertex')
    shortest_paths = get_shortest_paths(edges, point_of_interest)
    results = cudf.merge(shortest_paths, nodes[['vertex', 'y', 'x']], on='vertex', how='inner')
    polygons = get_polygons_for_travel_time(results, trip_times)
    del results, shortest_paths, edges, nodes
    return polygons, delayed(query_census_dataset)(polygons, census_data).compute()

In [9]:
%load_ext line_profiler

In [ ]:
%lprun -f get_data get_data(census_data.head(300_000))

## Dash App

In [ ]:
available_indicators = [25,35,45,60]
ns = Namespace("dlx", "choropleth")
ns_scatter = Namespace("dlx", "scatter")
classes = [0,1,2,3,4,5]
colorscale = ['#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C']
style = dict(weight=2, opacity=1, color='red', dashArray='3', fillOpacity=0.2)

minmax = dict(min=0, max=10)

geojson = dl.GeoJSON(id="scatter",data=get_data(census_data.head()),format="geojson",
                     zoomToBounds=True,  # when true, zooms to bounds when data changes
                     cluster=True,  # when true, data are clustered
                     clusterToLayer=ns_scatter("clusterToLayer"),  # how to draw clusters
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. cluster) on click
                     options=dict(pointToLayer=ns_scatter("pointToLayer")),  # how to draw points
                     superClusterOptions=dict(radius=150),  # adjust cluster size
                     hideout=dict(colorscale=colorscale, colorProp="income", **minmax)
                     )
chroma = "https://cdnjs.cloudflare.com/ajax/libs/chroma-js/2.1.0/chroma.min.js"

app = JupyterDash(__name__,external_scripts=[chroma],)
app.layout = html.Div([
    dl.Map([
           dl.TileLayer(), dl.LayerGroup(id="layer"),
           dl.GeoJSON(id='polygons', options=dict(style=ns("style")), hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="index"),),
           dl.GeoJSON(id="scatter",data=get_data(census_data.head()),format="geojson",
                     zoomToBounds=True,  # when true, zooms to bounds when data changes
                     cluster=True,  # when true, data are clustered
                     clusterToLayer=ns_scatter("clusterToLayer"),  # how to draw clusters
                     zoomToBoundsOnClick=True,  # when true, zooms to bounds of feature (e.g. cluster) on click
                     options=dict(pointToLayer=ns_scatter("pointToLayer")),  # how to draw points
                     superClusterOptions=dict(radius=150),  # adjust cluster size
                     hideout=dict(colorscale=colorscale, colorProp="income", **minmax)
                     ),
           ],
           id="map", zoom=2, center=(39, -100)
    ),
    html.Div([dcc.Dropdown(
                id='average-speed',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value=35
    )], style={"position": "relative", "bottom": "80px", "left": "10px", "z-index": "1000", "width": "200px"}),
    
], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block", "position": "relative"})

#  
@app.callback([Output("layer", "children"),Output("polygons", "data"), Output("scatter", "data")], [Input("map", "click_lat_lng"), Input('average-speed', 'value'),])
def map_click(click_lat_lng, average_speed):
    if click_lat_lng is not None:
        lat, long = click_lat_lng
        polygons, df = get_nearest_polygons_from_selected_point(lat, long, average_speed)
        d = gpd.geodataframe.from_shapely(polygons)
        polygon = gpd.GeoDataFrame(index=[0, 1,2, 3, 4], geometry=d).reset_index()
        return dl.Marker(position=click_lat_lng, children=dl.Tooltip("({:.3f}, {:.3f})".format(*click_lat_lng))),json.loads(polygon.to_json()), get_data(df.head(200_000))
    return None, None, None

app.run_server()

In [4]:
[i for i in range(1)]

[0]